In [ ]:
import pandas as pd
from datetime import datetime
import numpy as np
import lyricsgenius

genius = lyricsgenius.Genius("-VH73uIIYTXillAxkhh-uuEUfTNPXLVT25-EcyNlfBnXK2kSPidRXPQCsDumgKmw")

df = pd.DataFrame(pd.read_excel("./data/Hot 100 Audio Features.xlsx"))
df_hotstuff = pd.DataFrame(pd.read_csv("./data/Hot Stuff.csv"))

# drop songs without genres
df.dropna(subset=['spotify_genre'], inplace=True)

# get songs with rap genre
df_rap = pd.DataFrame()
for index, row in df.iterrows():
    genres = df.spotify_genre.squeeze()[index]
    if 'rap' in genres:
        df_rap = df_rap.append(row)

# drop duplicate songs (songs with same songID)
df_rap = df_rap.drop_duplicates(subset=['SongID'], keep='first')

# merge df_rap and df_hotstuff to get weekID
df_merge = pd.merge(df_rap, df_hotstuff, how='left')

# drop songs with no weekID
df_merge.dropna(subset=['WeekID'], inplace=True)

# drop duplicate songs (songs with same songID)
df_merge = df_merge.drop_duplicates(subset=['SongID'],keep='first')

# get years
years = []
for index, row in df_merge.iterrows():
    weekID = df_merge.WeekID[index]
    year = datetime.strptime(weekID, "%m/%d/%Y").year
    years.append(year)
df_merge['Year'] = years

# get final dataframe with year, performer, and song
df_final = df_merge[['Year', 'Performer', 'Song']]
df_final = df_final.reset_index(drop=True)

# get lyrics
glyrics = []
for index, row in df_final.iterrows():
    try:
        s = genius.search_song(row['Song'], row['Performer'])
        lyric = s.lyrics
    except:
        lyric = np.NaN
    glyrics.append(lyric)
df_final['Lyrics'] = glyrics

# drop rows with no lyrics
df_final.dropna(subset=['Lyrics'], inplace=True)

# export to csv
df_final.to_csv("Hot100Data3594.csv")